# OMICRON ANALYSIS

In [1]:
import pandas as pd
import requests
import io
from functools import reduce

## Gauteng Province (South Africa)

In [2]:
urls = {
    "Cases": "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv",
    "Deaths": "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_deaths.csv",
    "Hospitalisations": "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_raw_hospitalization.csv"
}

final_df = pd.DataFrame(pd.date_range(start='2020/03/05', end='2022/12/31'))
final_df.columns = ["Date"]

dfs = [final_df]
for category, url in urls.items():
    url = requests.get(url).content
    df = pd.read_csv(io.StringIO(url.decode('utf-8')), on_bad_lines='skip')
    if "Province" in df.columns:
        df = df[(df["Province"] == "Gauteng") & (df["Owner"] == "Total") & (df["variable"] == "AdmissionsinPreviousDay")]
        df = df[["Date", "value"]]
        df.columns = ["Date", category]
        df["Date"] =  pd.to_datetime(df["Date"])
    elif "GP" in df.columns:
        df = df[["date", "GP"]]
        df.columns = ["Date", category]
        df["Date"] =  pd.to_datetime(df["Date"] ,format = '%d-%m-%Y')
    dfs.append(df)
    df.to_csv(f"{category}.csv")

df_merged = reduce(lambda left, right: pd.merge(left, right, on = ["Date"], how='left'), dfs)
df_merged = df_merged.set_index("Date")
df_merged["Cases_diff"] = df_merged["Cases"].diff()
df_merged["Deaths_diff"] = df_merged["Deaths"].diff()
df_merged.to_csv("merged.csv")

In [3]:
df = pd.read_csv("merged.csv")

In [4]:
df_before = df[(df['Date'] < "2021-11-17")]
df_omicron = df[df['Date'] >= "2021-11-17"]

In [5]:
# Cases Per Death
cases_death_prior = int(df_before["Cases_diff"].mean()/df_before["Deaths_diff"].mean())
cases_death_omicron = int(df_omicron["Cases_diff"].mean()/df_omicron["Deaths_diff"].mean())

print("Cases/Death Prior: ", cases_death_prior)
print("Cases/Death Omicron: ", cases_death_omicron)

Cases/Death Prior:  45
Cases/Death Omicron:  313


In [6]:
# Cases Per Hospitalisation
hosp_cases_prior = int(df_before["Cases_diff"].mean()/df_before["Hospitalisations"].mean())
hosp_cases_omicron = int(df_omicron["Cases_diff"].mean()/df_omicron["Hospitalisations"].mean())
print("Cases/Hospitalisation Prior: ", hosp_cases_prior)
print("Cases/Hospitalisation Omicron: ", hosp_cases_omicron)

Cases/Hospitalisation Prior:  18
Cases/Hospitalisation Omicron:  51


In [7]:
# % Change
print("Change Hospitalisation", round((hosp_cases_prior-hosp_cases_omicron)/hosp_cases_omicron, 2)*100, "%")
print("Change Deaths", round((cases_death_prior-cases_death_omicron)/cases_death_omicron, 2)*100, "%")

Change Hospitalisation -65.0 %
Change Deaths -86.0 %


In [28]:
df

,Date,Cases,Deaths,Hospitalisations,Cases_diff,Deaths_diff
0,2020-03-05,0.0,NaN,NaN,NaN,NaN
1,2020-03-06,NaN,NaN,NaN,NaN,NaN
2,2020-03-07,1.0,NaN,NaN,NaN,NaN
3,2020-03-08,1.0,NaN,NaN,0.0,NaN
4,2020-03-09,1.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...
1027,2022-12-27,NaN,NaN,NaN,NaN,NaN
1028,2022-12-28,NaN,NaN,NaN,NaN,NaN
1029,2022-12-29,NaN,NaN,NaN,NaN,NaN
1030,2022-12-30,NaN,NaN,NaN,NaN,NaN


## England

In [29]:
url = "https://api.coronavirus.data.gov.uk/v2/data?areaType=nation&areaCode=E92000001&metric=newAdmissions&metric=newCasesBySpecimenDate&metric=newDeaths28DaysByDeathDate&format=csv"
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [30]:
df = df.drop(columns = ["areaCode", "areaType"])

In [31]:
df.columns = ["AreaName", "Date", "Hospitalisations", "Cases", "Deaths"]
df

,AreaName,Date,Hospitalisations,Cases,Deaths
0,England,2021-12-29,NaN,47037,24.0
1,England,2021-12-28,2082.0,117514,85.0
2,England,2021-12-27,1751.0,129647,73.0
3,England,2021-12-26,1374.0,86941,68.0
4,England,2021-12-25,1281.0,53838,49.0
...,...,...,...,...,...
695,England,2020-02-03,NaN,0,NaN
696,England,2020-02-02,NaN,0,NaN
697,England,2020-02-01,NaN,0,NaN
698,England,2020-01-31,NaN,0,NaN


In [15]:
omicron_cases = [22343, 16133, 40342, 19203, 13106, 14791, 6777, 11304, 9427, 3001, 1497, 4237, 519, 1534]
omicron_deaths = [4, 10, 10, 5, 6, 0, 2, 5, 6, 0, 0, 0, 0, 1]
print(sum(omicron_deaths))
omicron_hospitalisations = [98, 261, 41, 66, 105, 261, 25, 19, 20, 49, 5, 1, 0, 10]
print(195-sum(omicron_hospitalisations))


df["OmicronCases"] = pd.Series(omicron_cases)
df["OmicronDeaths"] = pd.Series(omicron_deaths)
df["OmicronHospitalisations"] = pd.Series(omicron_hospitalisations)

49
-766


In [16]:
print(len(omicron_hospitalisations))

14


In [17]:
df

,AreaName,Date,Hospitalisations,Cases,Deaths,OmicronCases,OmicronDeaths,OmicronHospitalisations
0,England,2021-12-28,NaN,37962,9.0,22343.0,4.0,98.0
1,England,2021-12-27,1751.0,114514,23.0,16133.0,10.0,261.0
2,England,2021-12-26,1374.0,85544,11.0,40342.0,10.0,41.0
3,England,2021-12-25,1281.0,53634,10.0,19203.0,5.0,66.0
4,England,2021-12-24,1020.0,102417,40.0,13106.0,6.0,105.0
...,...,...,...,...,...,...,...,...
694,England,2020-02-03,NaN,0,NaN,NaN,NaN,NaN
695,England,2020-02-02,NaN,0,NaN,NaN,NaN,NaN
696,England,2020-02-01,NaN,0,NaN,NaN,NaN,NaN
697,England,2020-01-31,NaN,0,NaN,NaN,NaN,NaN
